# Naive Bayes Algoritam

Da razjasnimo:

### 🔹 CategoricalNB
- radi sa **diskretnim kategorijama**
- očekuje vrednosti tipa: `0, 1, 2, ...`
- ne zanima ga redosled ni veličina

➡️ *„Koja kategorija je prisutna?“*

### 🔹 MultinomialNB
- radi sa **brojanjima**
- očekuje **koliko puta se nešto pojavilo**
- savršen za **TF / TF-IDF matrice**

➡️ *„Koliko puta se reč pojavila u dokumentu?“*


In [130]:
!mamba install pandas

mambajs 0.19.13

Specs: xeus-python, numpy, matplotlib, pillow, ipywidgets>=8.1.6, ipyleaflet, scipy, pandas, scikit-learn
Channels: emscripten-forge, conda-forge

Solving environment...
Solving took 1.4085999999977648 seconds
All requested packages already installed.


In [9]:
!mamba install scikit-learn

mambajs 0.19.13

Specs: xeus-python, numpy, matplotlib, pillow, ipywidgets>=8.1.6, ipyleaflet, scipy, pandas, scikit-learn
Channels: emscripten-forge, conda-forge

Solving environment...
Solving took 1.032199999999255 seconds
  Name                          Version                       Build                         Channel                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
+ brotli-python                 1.2.0                         py313h33caa6c_0               emscripten-forge              
+ certifi                       2026.1.4                      pyhd8ed1ab_0                  conda-forge                   
+ charset-normalizer            3.4.4                         pyhd8ed1ab_0                  conda-forge                   
+ idna                          3.11                          pyhd8ed1ab_0                  conda-forge                   
+ joblib                        1.5.3 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('balloons.csv')

In [5]:
df

,color,size,act,age,inflated
0,YELLOW,SMALL,STRETCH,ADULT,T
1,YELLOW,SMALL,STRETCH,ADULT,T
2,YELLOW,SMALL,STRETCH,CHILD,F
3,YELLOW,SMALL,DIP,ADULT,F
4,YELLOW,SMALL,DIP,CHILD,F
...,...,...,...,...,...
71,PURPLE,SMALL,DIP,CHILD,F
72,PURPLE,LARGE,STRETCH,ADULT,F
73,PURPLE,LARGE,STRETCH,CHILD,F
74,PURPLE,LARGE,DIP,ADULT,F


In [42]:
X = df.drop('inflated', axis=1) #ulazne
y = df['inflated']              #ciljna

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
# pisemo sa malim y!!!
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=123)

Pre korišćenja **CategoricalNB** modela potrebno je da se kategoricki atributi
sa $n_i$ kategorija mapiraju u numeričke vrednosti $0, 1, \ldots, n_i - 1$.

In [45]:
from sklearn.preprocessing import OrdinalEncoder

In [46]:
#oe = OrdinalEncoder() ovo radi na ispitu :)
oe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

Kao i obično, fit samo na trening skupu:

In [47]:
oe.fit(X_train)

,"categories categories: 'auto' or a list of array-like, default='auto'Categories (unique values) per feature:- 'auto' : Determine categories automatically from the training data.- list : ``categories[i]`` holds the categories expected in the ith column. The passed categories should not mix strings and numeric values, and should be sorted in case of numeric values.The used categories can be found in the ``categories_`` attribute.",'auto'
,"dtype dtype: number type, default=np.float64Desired dtype of output.",<class 'numpy.float64'>
,"handle_unknown handle_unknown: {'error', 'use_encoded_value'}, default='error'When set to 'error' an error will be raised in case an unknowncategorical feature is present during transform. When set to'use_encoded_value', the encoded value of unknown categories will beset to the value given for the parameter `unknown_value`. In:meth:`inverse_transform`, an unknown category will be denoted as None... versionadded:: 0.24",'use_encoded_value'
,"unknown_value unknown_value: int or np.nan, default=NoneWhen the parameter handle_unknown is set to 'use_encoded_value', thisparameter is required and will set the encoded value of unknowncategories. It has to be distinct from the values used to encode any ofthe categories in `fit`. If set to np.nan, the `dtype` parameter mustbe a float dtype... versionadded:: 0.24",-1
,"encoded_missing_value encoded_missing_value: int or np.nan, default=np.nanEncoded value of missing categories. If set to `np.nan`, then the `dtype`parameter must be a float dtype... versionadded:: 1.1",nan
,"min_frequency min_frequency: int or float, default=NoneSpecifies the minimum frequency below which a category will beconsidered infrequent.- If `int`, categories with a smaller cardinality will be considered infrequent.- If `float`, categories with a smaller cardinality than `min_frequency * n_samples` will be considered infrequent... versionadded:: 1.3 Read more in the :ref:`User Guide `.",None
,"max_categories max_categories: int, default=NoneSpecifies an upper limit to the number of output categories for each inputfeature when considering infrequent categories. If there are infrequentcategories, `max_categories` includes the category representing theinfrequent categories along with the frequent categories. If `None`,there is no limit to the number of output features.`max_categories` do **not** take into account missing or unknowncategories. Setting `unknown_value` or `encoded_missing_value` to aninteger will increase the number of unique integer codes by one each.This can result in up to `max_categories + 2` integer codes... versionadded:: 1.3 Read more in the :ref:`User Guide `.",None


In [48]:
X_train = oe.transform(X_train)
X_test = oe.transform(X_test)

In [49]:
from sklearn.naive_bayes import CategoricalNB

In [50]:
model = CategoricalNB()

In [51]:
model.fit(X_train, y_train)

,"alpha alpha: float, default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None
,"min_categories min_categories: int or array-like of shape (n_features,), default=NoneMinimum number of categories per feature.- integer: Sets the minimum number of categories per feature to `n_categories` for each features.- array-like: shape (n_features,) where `n_categories[i]` holds the minimum number of categories for the ith column of the input.- None (default): Determines the number of categories automatically from the training data... versionadded:: 0.24",None


In [52]:
model.category_count_

[array([[18., 11.],
        [ 8., 16.]]),
 array([[17., 12.],
        [ 6., 18.]]),
 array([[20.,  9.],
        [ 7., 17.]]),
 array([[11., 18.],
        [15.,  9.]])]

In [53]:
model.class_count_

array([29., 24.])

Na osnovu ove dve stvari računamo verovatnoću **P( y | X )** i predviđamo klasu.

In [54]:
y_train_pred = model.predict(X_train)

In [55]:
y_test_pred = model.predict(X_test)

In [58]:
from sklearn.metrics import confusion_matrix

In [59]:
confusion_matrix(y_train, y_train_pred)

array([[25,  4],
       [ 9, 15]], dtype=int64)

In [60]:
confusion_matrix(y_test, y_test_pred)

array([[9, 3],
       [3, 8]], dtype=int64)

-----------------------------------------

<h2> Klasifikacija teksta

<h3> TF matrica

In [62]:
import pandas as pd

In [63]:
data = pd.read_csv('20newsgroups.csv')

In [65]:
data.head()

,text,target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4


In [79]:
data.isna().any()

text       True
target    False
dtype: bool

In [86]:
data.dropna(inplace=True)

In [87]:
data.isna().any()

text      False
target    False
dtype: bool

1️⃣ **Šta u ovom skupu igra ulogu “teksta”?**

U 20 Newsgroups datasetu imaš:

- jednu kolonu sa **tekstom dokumenta** (često se zove text, content, message ili slično)

- jednu kolonu sa **kategorijom / grupom** (npr. target, group, label)

`TF matrica se pravi` **`ISKLJUČIVO`** `iz tekstualne kolone.`

2️⃣ **Ideja TF matrice (u jednoj rečenici)**

TF matrica broji **koliko se puta svaka reč pojavljuje u svakom dokumentu**.

- red = dokument

- kolona = reč

vrednost = broj pojavljivanja reči u tom dokumentu

4️⃣ **Formiranje TF matrice (NAJBITNIJI DEO)**

Za TF matricu se koristi CountVectorizer iz sklearn-a.

In [128]:
X = data['text']   # tekstualni dokumenti, ulazni 
y = data['target'] # ciljni

In [114]:
from sklearn.model_selection import train_test_split

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42,stratify=y)

In [116]:
from sklearn.feature_extraction.text import CountVectorizer

In [117]:
vectorizer = CountVectorizer()

In [118]:
X_train_tf = vectorizer.fit_transform(X_train)

In [119]:
X_test_tf  = vectorizer.transform(X_test)

<h4>Primena <b>MultinominalNB</b> nad TF:

In [120]:
from sklearn.naive_bayes import MultinomialNB

In [121]:
model = MultinomialNB()

In [122]:
model.fit(X_train_tf, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [123]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [127]:
y_pred = model.predict(X_test_tf)

cm  = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
f1  = f1_score(y_test, y_pred, average='macro')

#cm, acc, f1

<h3>TF-IDF matrica

> umesto CountVectorizer koristimo `TfidfVectorizer`

In [131]:
X = data['text']   # tekstualni dokumenti, ulazni 
y = data['target'] # ciljni

In [132]:
from sklearn.model_selection import train_test_split

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42,stratify=y)

In [135]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [136]:
vectorizer = TfidfVectorizer()

In [137]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [138]:
X_test_tfidf  = vectorizer.transform(X_test)

<h4>Primena <b>MultinominalNB</b> nad TF-IDF:

In [139]:
from sklearn.naive_bayes import MultinomialNB

In [140]:
model = MultinomialNB()

In [141]:
model.fit(X_train_tfidf, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [142]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [145]:
y_pred = model.predict(X_test_tfidf)

cm  = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
f1  = f1_score(y_test, y_pred, average='macro')

#cm, acc, f1